<a id="installation"></a>
# Installation

In [ ]:
!pip install -r scripts/requirements.txt

# Modules
import pandas as pd
import requests
from io import BytesIO
from zipfile import ZipFile
import matplotlib.pyplot as plt
import numpy as np
from lets_plot import *
from palmerpenguins import load_penguins
from urllib import request
from io import BytesIO
from sklearn.cluster import KMeans

# Fonctions
from scripts import import_eec

/opt/python/lib/python3.13/site-packages/palmerpenguins/penguins.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


## Import des données détail de l'enquête Emploi de 2018 à 2024

Je n'ai pas trouvé les données dans la base Melodi et l'Insee mais directement en ligne sur le site. J'ai essayé d'utiliser la fonction `pd.read_csv()` pour les télécharger directement depuis l'URL, mais les données sont souvent compressées au format .zip avec un autre fichier de documentation. J'ai rédigé une fonction `pd.read_csv_from_zip()` qui importe le plus gros fichier .csv contenu dans le dossier zippé au bout de l'URL. J'ai aussi ajouté fait une copie des données sur le cloud de mon compte SSPCloud et mis un lien "backup" qu'on peut indiquer à la fonction si le lien direct ne fonctionne pas (j'ai vu que c'était tout le temps fait comme ça dans le cours).

In [ ]:
# Liens vers les fichiers CSV accessibles directement depuis le site de l'Insee.
url_2024 = "https://www.insee.fr/fr/statistiques/fichier/8632441/FD_csv_EEC_2024.zip"
url_2023 = "https://www.insee.fr/fr/statistiques/fichier/8241122/FD_csv_EEC23.zip"
url_2022 = "https://www.insee.fr/fr/statistiques/fichier/7657353/FD_csv_EEC22.zip"
url_2021 = "https://www.insee.fr/fr/statistiques/fichier/6654604/FD_csv_EEC21.zip"
url_2020 = "https://www.insee.fr/fr/statistiques/fichier/5393560/fd_eec20_csv.zip"
url_2019 = "https://www.insee.fr/fr/statistiques/fichier/4809583/fd_eec19_csv.zip"
url_2018 = "https://www.insee.fr/fr/statistiques/fichier/4191029/fd_eec18_csv.zip"

# J'ai aussi enregistré les fichiers .csv non compressés dans un dossier de mon compte SSPCloud,
# normalement on devrait pouvoir y accéder sans problème à ces adresses s'il y a un problème avec les autres:
url_2024_backup = "https://minio.lab.sspcloud.fr/phobeika/open_eec/FD_EEC_2024.csv"
url_2023_backup = "https://minio.lab.sspcloud.fr/phobeika/open_eec/FD_csv_EEC23.csv"
url_2022_backup = "https://minio.lab.sspcloud.fr/phobeika/open_eec/FD_EEC_2022.csv"
url_2021_backup = "https://minio.lab.sspcloud.fr/phobeika/open_eec/FD_EEC_2021.csv"
url_2020_backup = "https://minio.lab.sspcloud.fr/phobeika/open_eec/FD_csv_EEC20.csv"
url_2019_backup = "https://minio.lab.sspcloud.fr/phobeika/open_eec/FD_EEC_2019.csv"
url_2018_backup = "https://minio.lab.sspcloud.fr/phobeika/open_eec/FD_csv_EEC18.csv"

# Si besoin, les dictionnaires de variables des différentes enquêtes sont à ces adresses (les variables ne sont pas les mêmes d'une année sur l'autre…) :
# url_doc_2018 = "https://www.insee.fr/fr/statistiques/fichier/4191029/EEC2018_dictionnaire_2020.pdf"
# url_doc_2019 = "https://www.insee.fr/fr/statistiques/fichier/4809583/EEC2019_dictionnaire_2020.pdf"
# url_doc_2020 = "https://www.insee.fr/fr/statistiques/fichier/5393560/EEC2020_Dictionnaire_Fichier_Detail.pdf"
# url_doc_2021 = "https://www.insee.fr/fr/statistiques/fichier/6654604/EEC%202021%20_%20Dictionnaire%20des%20codes%20_%20Fichier%20detail.pdf"
# url_doc_2022 = "https://www.insee.fr/fr/statistiques/fichier/7657353/EEC%202022%20_%20Dictionnaire%20des%20codes%20_%20Fichier%20detail.pdf"
# url_doc_2023 = "https://www.insee.fr/fr/statistiques/fichier/8241122/EEC%202023%20_%20Dictionnaire%20des%20codes%20_%20Fichier%20detail_2024_07_17.pdf"
# url_doc_2024 = "https://www.insee.fr/fr/statistiques/fichier/8632441/EEC%202024%20_%20Dictionnaire%20des%20codes%20_%20Fichier%20detail_2024.pdf"



In [5]:
# Import depuis le site de l'Insee :
eec24 = read_csv_from_zip(url_2024, backup_url = url_2024_backup, sep = ";")
eec23 = read_csv_from_zip(url_2023, backup_url = url_2023_backup, sep = ";")
eec22 = read_csv_from_zip(url_2022, backup_url = url_2022_backup, sep = ";")
eec21 = read_csv_from_zip(url_2021, backup_url = url_2021_backup, sep = ";")
eec20 = read_csv_from_zip(url_2020, backup_url = url_2020_backup, sep = ";")
eec19 = read_csv_from_zip(url_2019, backup_url = url_2019_backup, sep = ";")
eec18 = read_csv_from_zip(url_2018, backup_url = url_2018_backup, sep = ";")

Téléchargement depuis l'URL principale : https://www.insee.fr/fr/statistiques/fichier/8632441/FD_csv_EEC_2024.zip


Téléchargement depuis l'URL principale : https://www.insee.fr/fr/statistiques/fichier/8241122/FD_csv_EEC23.zip
Aucun mot-clé fourni : sélection du CSV le plus gros : FD_csv_EEC23.csv
Téléchargement depuis l'URL principale : https://www.insee.fr/fr/statistiques/fichier/7657353/FD_csv_EEC22.zip
Aucun mot-clé fourni : sélection du CSV le plus gros : FD_EEC_2022.csv
Téléchargement depuis l'URL principale : https://www.insee.fr/fr/statistiques/fichier/6654604/FD_csv_EEC21.zip
Aucun mot-clé fourni : sélection du CSV le plus gros : FD_EEC_2021.csv
Téléchargement depuis l'URL principale : https://www.insee.fr/fr/statistiques/fichier/5393560/fd_eec20_csv.zip
Aucun mot-clé fourni : sélection du CSV le plus gros : FD_csv_EEC20.csv
Téléchargement depuis l'URL principale : https://www.insee.fr/fr/statistiques/fichier/4809583/fd_eec19_csv.zip
Aucun mot-clé fourni : sélection du CSV le plus gros : FD_EEC_2019.csv
Téléchargement depuis l'URL principale : https://www.insee.fr/fr/statistiques/fichier/41

In [5]:
# Variables qu'on voudrait garder (dictionnaire des codes 2018)
# - ACTEU : activité au sens du BIT (1: actifs occupés 2: chômeurs 3: inactifs)
# - AGE5 : je crois que c'est le plus détaillé au niveau de l'âge : il y a 5 classes d'âge - devient AGE6 en 2021
# - ANCENTR4 : ancienneté dans l'entreprise ou dans la fonction publique, 4 postes
# - ANNEE : année de référence de l'enquête
# - CATAU2010R : catégorie de la commune (pas sûr de comprendre les modalités)
# - CHPUB : nature de l'employeur principal
# - CSA : catégorie socioprofessionnelle pour les inactifs (2 chiffres)
# - CSE : catégorie socioprofessionnelle pour les actifs (2 chiffres)
# - CSP : catégorie socioprofessionnelle pour les actifs occupés (2 chiffres)
# - DIP11 : diplôme le plus élevé obtenu
# - DUHAB : type de temps de travail
# - EMPNBH : nombre d'heures effectuées dans l'emploi principal pendant la semaine de référence
# - EXTRIAN : pondération
# - FONCTC : fonction de la profession
# - HHC6 : nombre d'heures travaillées en moyenne dans l'emploi principal
# - JOURTR : nombre de jours travaillés en général par semaine
# - NAFANT : nature de l'activité de l'emploi antérieur (pour chômeurs et inactifs)
# - NAFG021UN : nature de l'activité de l'emploi principale (NAF2 à 21 postes)
# - NONDIC : raison de la non disponibilité pour travailler sous deux semaines
# - PASTRA : n'a pas travaillé la semaine de référence mais a un emploi rémunéré
# - PUB3FP : statut public ou privé de l'employeur principal
# - QPRC : position professionnelle dans l'emploi principal
# - RABS : raison de l'absence au travail la semaine de référence, pour les personnes qui ont un emploi rémunéré (2 : congé maladie ou accident du travail)
# - RAISNREC : raison principale de la non recherche d'emploi durant les quatre semaines s'achevant par la semaine de référence (4 : problèmes de santé)
# - RAISNSOU : raison pour laquelle l'individu ne souhaite pas travaille actuellement (3 : a des problèmes de santé)
# - RAISON : raison de la non-disponibilité pour travailler davantage (pour ceux qui souhaitent faire plus d'heures de travail) (4 : pour raisons de santé)
# - SEXE
# - SOUA : souhait d'avoir un emploi en remplacement de l'emploi actuel ou à venir
# - STAT2 : salarié (2) ou non salarié (1)
# - TPPRED : temps de travail dans l'emploi principal, redressé et imputé de la non-réponse partielle
# - TRAREF : exercice d'au moins une heure de travail durant la semaine de référence
# - TRIM : trimestre de référence de l'enquête (1 à 4)

vars = ['ACTEU', 
        'AGE5', 
        'ANCENTR4', 
        'ANNEE', 
        'CATAU2010R', 
        'CHPUB', 
        'CSA', 
        'CSE', 
        'CSP', 
        'DIP11', 
        'DUHAB', 
        'EMPNBH', 
        'EXTRIAN', 
        'FONCTC', 
        'HHC6', 
        'JOURTR', 
        'NAFANT', 
        'NAFG021UN',
        'NONDIC', 
        'PASTRA', 
        'PUB3FP', 
        'QPRC', 
        'RABS', 
        'RAISNREC', 
        'RAISNSOU', 
        'RAISON',
        'SEXE',
        'SOUA',
        'STAT2',
        'TPPRED',
        'TRAREF',
        'TRIM']

# Ça marche pour 2018, 2019 et 2020. Ensuite beaucoup de variables changent de nom (ou plus)

# Assemblage des tables des différentes années

Les variables présentes dans les différentes tables ne sont pas les mêmes d'année en année. Par exemple, la variable d'âge est en 5 modalités et s'appelle `AGE5` en 2018 et 2019 puis elle passe à 6 modalités avec un découpage différent à partir de 2020 et s'appelle alors `AGE6`. Dans un premier temps, on peut ne conserver que les variables qui sont présentes dans toutes les tables, en espérant que les modalités restent les mêmes la plupart du temps. Si on veut en utiliser d'autres, il faudra ajouter des recodages préalables.

In [6]:
# On crée une liste de tous tes DataFrames
dfs = [eec18, eec19, eec20, eec21, eec22, eec23, eec24]

# On calcule l'intersection des colonnes
vars_communes = set(dfs[0].columns)
for df in dfs[1:]:
    vars_communes = vars_communes.intersection(df.columns)

# On convertit en liste triée pour lecture
vars_communes = sorted(vars_communes)
print(f"{len(vars_communes)} variables communes :")
print(vars_communes)

33 variables communes :
['AAC', 'ACTEU', 'ANCCHOM', 'ANNEE', 'CHPUB', 'DISPPLC', 'ENFRED', 'EXTRIAN', 'HALOR', 'METRODOM', 'NAFANTG004N', 'NAFG004UN', 'NAFG010UN', 'NAFG017UN', 'NAFG021UN', 'NAFG038UN', 'NAFG088UN', 'OFFICC', 'PASTRA', 'PASTRB', 'PUB3FP', 'RABS', 'RAISTP', 'SEXE', 'SOUSEMPL', 'STATUT', 'STC', 'STPLC', 'TEMP', 'TPPRED', 'TRAREF', 'TRIM', 'TXTPPRED']


Il y a donc 33 variables qui sont présentes avec le même nom dans toutes les bases de 2018 à 2024. Pour ne pas s'encombrer avec des variables qu'on utilise pas, on peut faire une liste des variables qu'on veut utiliser parmi celles-ci, qu'on peut allonger selon les besoins.

In [7]:
# Liste des variables qu'on veut utiliser
core_vars = ['ACTEU', 'RABS', 'SEXE', 'ANNEE', 'EXTRIAN']
# Concaténation des différents fichiers dans une seule tablea "eec_all"
eec_all = pd.concat(
    [eec18[core_vars], eec19[core_vars], eec20[core_vars], eec21[core_vars], eec22[core_vars], eec23[core_vars]],
    ignore_index = True)


In [ ]:
# En fait on ne garde que les actifs en emploi (`ACTEU` == 1)
eec_actifs = eec_all[eec_all['ACTEU'] == 1]
eec_actifs['RABS'] = eec_actifs['RABS'].astype(str)

/tmp/ipykernel_99363/4222421430.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eec_actifs['RABS'] = eec_actifs['RABS'].astype(str)


13

In [54]:
ggplot(eec_all, aes(x = 'SEXE') + geom_bar())